In [150]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import optuna
import re

In [152]:

df=pd.read_csv('spam.csv')
df=df.loc[:,['v1','v2']]
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [20]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [154]:
port_Stem=PorterStemmer()
def streming(content):
    stremed_content=re.sub('[^a-zA-Z]',' ',content)
    stremed_content=stremed_content.lower()
    stremed_content=stremed_content.split()
    stremed_content=[port_Stem.stem(word) for word in stremed_content if not word in stopwords.words('english')]
    stremed_content=' '.join(stremed_content)
    return stremed_content

In [155]:
df['v2']=df['v2'].apply(streming)

In [153]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['v1']=le.fit_transform(df.loc[:,'v1'])

In [146]:
df.head()

,v1,v2
0,0,go jurong point crazi avail bugi n great world...
1,0,ok lar joke wif u oni
2,1,free entri wkli comp win fa cup final tkt st m...
3,0,u dun say earli hor u c alreadi say
4,0,nah think goe usf live around though


In [156]:
y=df.loc[:,'v1'].values
x=df.loc[:,'v2'].values

In [157]:
# converting textual data to numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(x)
x=vectorizer.transform(x)

In [136]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)

In [94]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [137]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

In [138]:
estimators={'lr':LogisticRegression(),'svc':SVC(),'rfc':RandomForestClassifier(),'xgc':XGBClassifier(),'adc':AdaBoostClassifier()}

In [139]:
for name,model in estimators.items():
    model.fit(x_train,y_train)
    print("model",model)
    y_pred=model.predict(x_test)
    score=accuracy_score(y_pred,y_test)
    print("accuracy score:",score)

model LogisticRegression()
accuracy score: 0.9504666188083274
model SVC()
accuracy score: 0.9720028715003589
model RandomForestClassifier()
accuracy score: 0.9770279971284996
model XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)
accuracy score: 0.9727207465900933


c:\Users\shish\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


model AdaBoostClassifier()
accuracy score: 0.9669777458722182


In [99]:
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, x_train, y_train, cv=3, scoring='accuracy').mean()

    return score

In [100]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-03-05 17:31:22,251] A new study created in memory with name: no-name-22ae8155-96d2-484b-b483-1b48a8e8871f
[I 2025-03-05 17:31:22,800] Trial 0 finished with value: 0.8671931083991385 and parameters: {'n_estimators': 104, 'max_depth': 4}. Best is trial 0 with value: 0.8671931083991385.
[I 2025-03-05 17:31:23,335] Trial 1 finished with value: 0.8801148600143575 and parameters: {'n_estimators': 68, 'max_depth': 7}. Best is trial 1 with value: 0.8801148600143575.
[I 2025-03-05 17:31:24,133] Trial 2 finished with value: 0.893993778415889 and parameters: {'n_estimators': 89, 'max_depth': 9}. Best is trial 2 with value: 0.893993778415889.
[I 2025-03-05 17:31:24,679] Trial 3 finished with value: 0.8904044029672171 and parameters: {'n_estimators': 61, 'max_depth': 9}. Best is trial 2 with value: 0.893993778415889.
[I 2025-03-05 17:31:26,748] Trial 4 finished with value: 0.9514237855946398 and parameters: {'n_estimators': 118, 'max_depth': 20}. Best is trial 4 with value: 0.95142378559463

In [160]:
rfc=RandomForestClassifier()
rfc.fit(x_train,y_train)
y_pred=rfc.predict(x_test)
score=accuracy_score(y_pred,y_test)
score

0.9763101220387652

In [159]:
import pickle
pickle.dump(rfc,open("rfc.pkl","wb"))
pickle.dump(vectorizer,open("tfd.pkl","wb"))

In [105]:
# load model
rfc = pickle.load(open('rfc.pkl','rb'))

In [164]:
def helper(text):
    test=streming(text)
    test=np.array([test])
    test=vectorizer.transform(test)
    pred=rfc.predict(test)
    return pred[0]

In [165]:
helper("hello my name is google and im here to inform you about your placements")

0

In [111]:
rfc.predict([['hello im a fish']])

ValueError: could not convert string to float: 'hello im a fish'